# **Import Library**

In [0]:
import pandas as pd
import numpy as np

# **Import File**

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df_one = pd.read_csv('/content/drive/My Drive/work/nectec/dataset7.csv',parse_dates=['Date'])
with pd.ExcelFile('/content/drive/My Drive/work/nectec/dataSet4 (2)-newFromITaaS.xlsx') as reader:
  # print(reader.sheet_names)
  df_two = pd.read_excel(reader, sheet_name=reader.sheet_names[1])


In [24]:
def clean_data_edit(df_one,df_two):
  df_all_columns = pd.merge(df_one, df_two, on='Item No', how='inner')
  df_all_columns['sum_price'] = df_all_columns['Price'] * df_all_columns['Quantity']
  df_clean_all = df_all_columns[['Receipt No','Item No','Price','Quantity','Date','Description','sum_price']]
  # edit_data
  length_all_data = int(len(df_clean_all)/3)
  df_clean_all['year'] = 2018
  df_clean_all['year'][length_all_data:length_all_data*2] = 2019
  df_clean_all['year'][length_all_data*2:length_all_data*3] = 2020
  df_clean_all['month'] = df_clean_all['Date'].dt.month
  df_clean_all['date'] = df_clean_all['Date'].dt.day
  df_clean_all['quarter'] = df_clean_all['Date'].dt.quarter
  return df_clean_all
df = clean_data_edit(df_one,df_two)
df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1240: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_lau

,Receipt No,Item No,Price,Quantity,Date,Description,sum_price,year,month,date,quarter
0,140050000,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
1,140050221,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
2,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
3,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
4,130031218,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
981540,30020755,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0,2020,9,30,3
981541,40068855,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0,2020,9,30,3
981542,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0,2020,9,30,3
981543,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0,2018,9,30,3


# **Function Bar Chart**

In [25]:
from calendar import monthrange
import plotly.graph_objects as go
def virtualize_bar_chart(df,start_date,end_date,category_date,category_stock):
    df_sum_year = df.groupby(['year','month','date','Date']).sum().reset_index()
    df_sum_year = df_sum_year[['year','month','date','Price','Quantity','Date']]
    list_unique_year = df_sum_year['year'].unique().tolist()

    list_year = []
    list_month = []
    list_all_category = []
    list_price = []
    list_quantity = []
    list_all_date = []
    list_date = []
    list_unique_month = [1,2,3,4,5,6,7,8,9,10,11,12]
    for i in range(len(list_unique_year)):
      for j in range(len(list_unique_month)):
        num_days = monthrange(int(list_unique_year[i]), int(list_unique_month[j]))[1]
        for k in range(num_days):
          if k+1 not in df_sum_year[(df_sum_year['year'] == list_unique_year[i]) & (df_sum_year['month'] == list_unique_month[j])]['date'].unique().tolist():
              list_year.append(list_unique_year[i])
              list_month.append(list_unique_month[j])
              list_all_category.append(k+1)
              list_price.append(0)
              list_quantity.append(0)
        
    df_not_data = pd.DataFrame({'year':list_year,'month':list_month,'date':list_all_category,'Price':list_price,'Quantity':list_quantity})
    clean_data = pd.concat([df_sum_year,df_not_data]).reset_index()
    for i in range(len(clean_data)):
      list_date.append(f"{clean_data['year'][i]}-{clean_data['month'][i]}-{clean_data['date'][i]}")
    clean_data['Date'] = pd.to_datetime(list_date)
    df_clean_choose = clean_data.sort_values(['Date'], ascending=[True])
    df_clean_choose['week'] = df_clean_choose['Date'].dt.dayofweek
    df_between_date = df_clean_choose[(df_clean_choose['Date'] > start_date) & (df_clean_choose['Date'] <= end_date)]
    df_between_date['quarter'] = df_between_date['Date'].dt.quarter


    # 
    list_price = []
    list_quantity = []

    if 'year' == category_date:
        df_sum = df_between_date.groupby(['year']).sum().reset_index()[['year','Price','Quantity']]
        list_name = df_sum['year'].tolist()
        list_price_category = df_sum['Price'].tolist() 
        list_quantity_category = df_sum['Quantity'].tolist()
        for i in range(len(list_name)):
            list_price.append({'x': [list_name[i]], 'y': [list_price_category[i]], 'type': 'bar', 'name': '{}'.format(list_name[i])})
            list_quantity.append({'x': [list_name[i]], 'y': [list_quantity_category[i]], 'type': 'bar', 'name': '{}'.format(list_name[i])})

    if 'quarter' == category_date or 'month' == category_date or 'date' == category_date:
        df_sum = df_between_date.groupby(['year',f'{category_date}']).sum().reset_index()[['year',f'{category_date}','Price','Quantity']].sort_values(by=[f'{category_date}']).reset_index()
        list_name_year = df_sum['year'].tolist()
        list_name = df_sum[f'{category_date}'].tolist()
        list_price_category = df_sum['Price'].tolist()
        list_quantity_category = df_sum['Quantity'].tolist()
        df_list_quarter = pd.DataFrame({
            'year':list_name_year,
            'name':list_name,
            'list_price_category':list_price_category,
            'list_quantity_category':list_quantity_category
        }).sort_values(by='year')
        # print(df_list_quarter)
        list_unique_year = df_list_quarter['year'].unique().tolist()
        for i in range(len(list_unique_year)):
            list_price.append({'x': df_list_quarter[df_list_quarter['year'] == list_unique_year[i]]['name'].tolist(), 'y': df_list_quarter[df_list_quarter['year'] == list_unique_year[i]]['list_price_category'].tolist(), 'type': 'bar', 'name': f'{list_name_year[i]}'})
            list_quantity.append({'x': df_list_quarter[df_list_quarter['year'] == list_unique_year[i]]['name'].tolist(), 'y': df_list_quarter[df_list_quarter['year'] == list_unique_year[i]]['list_quantity_category'].tolist(), 'type': 'bar', 'name': f'{list_name_year[i]}'})
    elif 'week' == category_date:
        df_sum_week = df_between_date.groupby(['year','week']).sum().reset_index()[['year','week','Price','Quantity']].sort_values(by=['week']).reset_index()
        dict_week = {0:'monday',
                    1:'Tuesday',
                    2:'Wednesday',
                    3:'Thursday',
                    4:'Friday',
                    5:'Saturay',
                    6:'Sunday',}
        list_week_name = []
        for i in range(len(df_sum_week)):
            list_week_name.append(dict_week[df_sum_week['week'][i]])
        list_name_year_week = df_sum_week['year'].tolist()
        list_price_week = df_sum_week['Price'].tolist()
        list_quantity_week = df_sum_week['Quantity'].tolist()

        df_list_week = pd.DataFrame({
            'year':list_name_year_week,
            'name':list_week_name,
            'list_price_category':list_price_week,
            'list_quantity_category':list_quantity_week
        }).sort_values(by='year')
        list_unique_year = df_list_week['year'].unique().tolist()
        for i in range(len(list_unique_year)):
            list_price.append({'x': df_list_week[df_list_week['year'] == list_unique_year[i]]['name'].tolist(), 'y': df_list_week[df_list_week['year'] == list_unique_year[i]]['list_price_category'].tolist(), 'type': 'bar', 'name': '{}'.format(list_name_year_week[i])})
            list_quantity.append({'x': df_list_week[df_list_week['year'] == list_unique_year[i]]['name'].tolist(), 'y': df_list_week[df_list_week['year'] == list_unique_year[i]]['list_quantity_category'].tolist(), 'type': 'bar', 'name': '{}'.format(list_name_year_week[i])})
    if category_stock == 'price':
        solution = list_price
    elif category_stock == 'quantity':
        solution = list_quantity
    fig = go.Figure(data=solution,layout_title_text=f"{category_date}_{category_stock}")
    fig.update_layout(barmode='group')
    return fig.show()
virtualize_bar_chart(df,'2018-01-12','2020-12-18','week','price')
virtualize_bar_chart(df,'2018-01-12','2020-12-18','quarter','quantity')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
df

,Receipt No,Item No,Price,Quantity,Date,Description,sum_price,year,month,date,quarter
0,140050000,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
1,140050221,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
2,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
3,80004023,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
4,130031218,3239082A,35.0,1.0,2018-01-01,ข้าวเม่าทรงเครื่องรสดั้งเดิม-ต้นกล้า,35.0,2018,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
981540,30020755,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0,2020,9,30,3
981541,40068855,2751012A,40.0,1.0,2018-09-30,น้ำพริกตาแดงเห็ด3อย่าง(เจ)-แย้มเหน่อ,40.0,2020,9,30,3
981542,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0,2020,9,30,3
981543,40068655,3412173A,1250.0,1.0,2018-09-30,เสื้อลินินคอปาดผูกโบว์ชาย-ริมนที,1250.0,2018,9,30,3




```
`# This is formatted as code`
```

# **Function Sunburst Chart**

In [27]:
from calendar import monthrange

def data_surburst(df,start_date,end_date):

  clean_data = df.groupby(['Receipt No','year','month','date','Date']).count().reset_index()[['Receipt No','year','month','date','Date','Item No']]
  clean_data = clean_data.rename(columns={"Item No": "count_receipt"})
  clean_data

  # list_date = []

  # for i in range(len(clean_data[:5])):
  #   list_date.append(f"{clean_data['year'][i]}-{clean_data['month'][i]}-{clean_data['date'][i]}")


  df_clean_choose = clean_data.sort_values(['Date'], ascending=[True])
  df_clean_choose['week'] = df_clean_choose['Date'].dt.dayofweek
  df_between_date = df_clean_choose[(df_clean_choose['Date'] > start_date) & (df_clean_choose['Date'] <= end_date)]
  df_between_date['quarter'] = df_between_date['Date'].dt.quarter
  df_between_date = df_between_date.reset_index()
  many_receipt = df_between_date[df_between_date['count_receipt'] > 1]
  many_receipt['category_receipt'] = 'many_sku'
  one_receipt = df_between_date[df_between_date['count_receipt'] == 1]
  one_receipt['category_receipt'] = 'one_sku'
  df_sum = pd.concat([many_receipt,one_receipt])
  df_sum = df_sum.sort_index()
  return df_sum
df_data_surburst = data_surburst(df,'2018-01-01','2019-12-18')[:2000]
# print(df_data_surburst)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [31]:
import plotly.graph_objects as go
from calendar import monthrange
def filter_dataframe(catagory_date,filter_date_level,receipt_no):

  if 'year' == catagory_date:
    filter_category = df_data_surburst[(df_data_surburst['year'] == filter_date_level) & (df_data_surburst['Receipt No'] == receipt_no)]['count_receipt'].sum()
  elif 'quarter' == catagory_date:
    filter_category = df_data_surburst[(df_data_surburst['year-quarter'] == filter_date_level) & (df_data_surburst['Receipt No'] == receipt_no)]['count_receipt'].sum()
  elif 'month' == catagory_date:
    filter_category = df_data_surburst[(df_data_surburst['year-quarter-month'] == filter_date_level) & (df_data_surburst['Receipt No'] == receipt_no)]['count_receipt'].sum()
  elif 'week' == catagory_date:
    filter_category = df_data_surburst[(df_data_surburst['year-quarter-month-week'] == filter_date_level) & (df_data_surburst['Receipt No'] == receipt_no)]['count_receipt'].sum()
  elif 'date' == catagory_date:
   filter_category = df_data_surburst[(df_data_surburst['year-month-day'] == filter_date_level) & (df_data_surburst['Receipt No'] == receipt_no)]['count_receipt'].sum()
  return filter_category


def virtualize_surburst_chart(df,category_date):
  df_sum = df_data_surburst
  df_sum['year-quarter'] = df_sum['year'].map(lambda x: str(x) + '_') + df_sum['quarter'].map(lambda x: str(x))
  df_sum['year-quarter-month'] = df_sum['year'].map(lambda x: str(x) + '_') + df_sum['quarter'].map(lambda x: str(x) + '_') + df_sum['month'].map(lambda x: str(x))
  df_sum['year-quarter-month-week'] = df_sum['year'].map(lambda x: str(x) + '_') + df_sum['quarter'].map(lambda x: str(x) + '_') + df_sum['month'].map(lambda x: str(x) + '_') + df_sum['week'].map(lambda x: str(x))
  df_sum['year-month'] = df_sum['year'].map(lambda x: str(x) + '_') + df_sum['month'].map(lambda x: str(x))
  df_sum['year-month-day'] = df_sum['year'].map(lambda x: str(x) + '_') + df_sum['month'].map(lambda x: str(x) + '_') + df_sum['date'].map(lambda x: str(x))
  sunburst_origin = [('total_sku', np.nan)]
  if 'year' == category_date:
    label_year = df_sum['year'].unique().tolist()
    parent_year = ["total_sku" for i in range(len(label_year))]

    df_surburst_year = pd.DataFrame({
                            'label': label_year,
                            'parent': parent_year
                            })
    surburst_year = list(zip(df_surburst_year['label'], df_surburst_year['parent']))

    df_sunburst_year_receipt_no = pd.DataFrame({
                            'label': list(df_sum['Receipt No']),
                            'parent': list(df_sum['year'])
                            })

    sunburst_year_receipt = list(set(list(zip(df_sunburst_year_receipt_no['label'], df_sunburst_year_receipt_no['parent']))))
    sunburst_year_receipt
    return pd.DataFrame(sunburst_origin + surburst_year + sunburst_year_receipt, columns=['label', 'parent'])
  elif 'quarter' == category_date:
    label_year = df_sum['year'].unique().tolist()
    parent_year = ["total_sku" for i in range(len(label_year))]
    df_surburst_year = pd.DataFrame({
                        'label': label_year,
                        'parent': parent_year,
                        
                        })
    surburst_year = list(set(list(zip(df_surburst_year['label'], df_surburst_year['parent']))))
    df_surburst_year_quarter = pd.DataFrame({
        'label': list(df_sum['year-quarter']),
        'parent': list(df_sum['year'])
    })
    surburst_year_quarter = list(set(list(zip(df_surburst_year_quarter['label'],df_surburst_year_quarter['parent']))))
    df_surburst_quarter_receipt = pd.DataFrame({
        'label': list(df_sum['Receipt No']),
        'parent': list(df_sum['year-quarter'])
    })
    surburst_quarter_receipt = list(set(list(zip(df_surburst_quarter_receipt['label'],df_surburst_quarter_receipt['parent']))))
    all_surburst = sunburst_origin + surburst_year + surburst_year_quarter + surburst_quarter_receipt
    return pd.DataFrame(all_surburst, columns=['label', 'parent'])

  elif 'month' == category_date:
    label_year = df_sum['year'].unique().tolist()
    parent_year = ["total_sku" for i in range(len(label_year))]
    df_surburst_year = pd.DataFrame({
                        'label': label_year,
                        'parent': parent_year
                        })
    surburst_year = list(set(list(zip(df_surburst_year['label'], df_surburst_year['parent']))))
    df_surburst_year_quarter = pd.DataFrame({
        'label': list(df_sum['year-quarter']),
        'parent': list(df_sum['year'])
    })
    surburst_year_quarter = list(set(list(zip(df_surburst_year_quarter['label'],df_surburst_year_quarter['parent']))))

    df_surburst_year_quarter_month = pd.DataFrame({
        'label': list(df_sum['year-quarter-month']),
        'parent': list(df_sum['year-quarter'])
    })

    surburst_year_quarter_month = list(set(list(zip(df_surburst_year_quarter_month['label'],df_surburst_year_quarter_month['parent']))))

    df_surburst_month_receipt = pd.DataFrame({
        'label': list(df_sum['Receipt No']),
        'parent': list(df_sum['year-quarter-month'])
    })
    surburst_month_receipt = list(set(list(zip(df_surburst_month_receipt['label'],df_surburst_month_receipt['parent']))))
    all_surburst = sunburst_origin + surburst_year + surburst_year_quarter + surburst_year_quarter_month + surburst_month_receipt
    return pd.DataFrame(all_surburst, columns=['label', 'parent'])
  elif 'week' == category_date:
    label_year = df_sum['year'].unique().tolist()
    parent_year = ["total_sku" for i in range(len(label_year))]
    df_surburst_year = pd.DataFrame({
                        'label': label_year,
                        'parent': parent_year
                        })
    surburst_year = list(set(list(zip(df_surburst_year['label'], df_surburst_year['parent']))))
    df_surburst_year_quarter = pd.DataFrame({
        'label': list(df_sum['year-quarter']),
        'parent': list(df_sum['year'])
    })
    surburst_year_quarter = list(set(list(zip(df_surburst_year_quarter['label'],df_surburst_year_quarter['parent']))))

    df_surburst_year_quarter_month = pd.DataFrame({
        'label': list(df_sum['year-quarter-month']),
        'parent': list(df_sum['year-quarter'])
    })

    surburst_year_quarter_month = list(set(list(zip(df_surburst_year_quarter_month['label'],df_surburst_year_quarter_month['parent']))))

    df_surburst_year_quarter_month_week = pd.DataFrame({
        'label': list(df_sum['year-quarter-month-week']),
        'parent': list(df_sum['year-quarter-month'])
    })

    surburst_year_quarter_month_week = list(set(list(zip(df_surburst_year_quarter_month_week['label'],df_surburst_year_quarter_month_week['parent']))))

    df_surburst_week_receipt = pd.DataFrame({
        'label': list(df_sum['Receipt No']),
        'parent': list(df_sum['year-quarter-month-week'])
    })
    surburst_week_receipt = list(set(list(zip(df_surburst_week_receipt['label'],df_surburst_week_receipt['parent']))))
    all_surburst = sunburst_origin + surburst_year + surburst_year_quarter + surburst_year_quarter_month + surburst_year_quarter_month_week + surburst_week_receipt
    return pd.DataFrame(all_surburst, columns=['label', 'parent'])
    
  elif 'day' == category_date:
    label_year = df_sum['year'].unique().tolist()
    parent_year = ["total_sku" for i in range(len(label_year))]
    df_surburst_year = pd.DataFrame({
                        'label': label_year,
                        'parent': parent_year
                        })
    surburst_year = list(set(list(zip(df_surburst_year['label'], df_surburst_year['parent']))))

    df_surburst_year_month = pd.DataFrame({
        'label': list(df_sum['year-month']),
        'parent': list(df_sum['year'])
    })
    surburst_year_month = list(set(list(zip(df_surburst_year_month['label'], df_surburst_year_month['parent'])))) 

    df_surburst_year_month_day = pd.DataFrame({
        'label': list(df_sum['year-month-day']),
        'parent': list(df_sum['year-month'])
    })
    surburst_year_month_day = list(set(list(zip(df_surburst_year_month_day['label'], df_surburst_year_month_day['parent'])))) 

    df_surburst_day_receipt = pd.DataFrame({
        'label': list(df_sum['Receipt No']),
        'parent': list(df_sum['year-month-day'])
    })
    surburst_day_receipt = list(set(list(zip(df_surburst_day_receipt['label'],df_surburst_day_receipt['parent']))))
    all_surburst = sunburst_origin + surburst_year + surburst_year_month + surburst_year_month_day + surburst_day_receipt

    return pd.DataFrame(all_surburst, columns=['label', 'parent'])
def show_sunburst(category_date):
  df_sunburst = virtualize_surburst_chart(df[:300],f'{category_date}')

  df_list = list(zip(df_sunburst['label'], df_sunburst['parent']))
  df_list
  df_sunburst['qty'] = list(map(lambda row: filter_dataframe(f'{category_date}', row[1], row[0]), df_list))

  fig = go.Figure()

  fig.add_trace(go.Sunburst(
      # ids=df.index(),
      labels=df_sunburst.label,
      parents=df_sunburst.parent,
      domain=dict(column=0),
      values=df_sunburst.qty,
  ))


  fig.update_layout(
      grid= dict(columns=2, rows=1),
      margin = dict(t=0, l=0, r=0, b=0)
  )

  return fig.show()
show_sunburst('date')


TypeError: ignored

# **Function Venn Diagram**

In [29]:
list_date = []
for i in range(len(df)):
  list_date.append(f"{df['year'][i]}-{df['month'][i]}-{df['date'][i]}")
df['Date'] = pd.to_datetime(list_date)

df_group_receipt = df.groupby(['Receipt No','year']).count().reset_index()
df_group_receipt = df_group_receipt[['Receipt No','year']]
list_unique_receipt = df_group_receipt['Receipt No'].unique().tolist()
df_receipt_all = df_group_receipt.groupby('Receipt No')
print(df_receipt_all)


In [30]:
list_all_duplicate = []
list_not_duplicate = []
for i in range(len(list_unique_receipt[:2000])):
  list_year_unique = df_receipt_all.get_group(list_unique_receipt[i])['year'].unique().tolist()
  list_all_duplicate.append(f"{list_year_unique}")

dict_sep_year = dict((i, list_all_duplicate.count(i)) for i in list_all_duplicate)
list_unique_year = df_between_date['year'].unique().tolist()
tuple_unique_year = tuple(df_between_date['year'].unique())

list_set_year = []
for i in range(len(list_unique_year)):
  list_set_year.append([list_unique_year[i]])
  for j in range(len(list_unique_year)):
    if i != j and i < j:
      list_set_year.append([list_unique_year[i],list_unique_year[j]])
      for k in range(len(list_unique_year)):
        if i != j and i !=k and j != k and i < k and i < j and j < k:
          list_set_year.append([list_unique_year[i],list_unique_year[j],list_unique_year[k]])
list_set_year.sort(key = lambda x: x[0])
list_set_year = sorted(list_set_year, key=len)
list_set_year

NameError: ignored

In [0]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import numpy as np

def virtualize_surburst_chart(df,start_date,end_date):

    df['Date'] = pd.to_datetime(list_date)
    df_between_date = df[(df['Date'] > start_date) & (df['Date'] <= end_date)]
    df_between_date['quarter'] = df_between_date['Date'].dt.quarter
    df_group_receipt = df_between_date.groupby(['Receipt No','year']).count().reset_index()
    df_group_receipt = df_group_receipt[['Receipt No','year']]
    list_unique_receipt = df_group_receipt['Receipt No'].unique().tolist()
    df_receipt_all = df_group_receipt.groupby('Receipt No')
    list_all_duplicate = []
    list_not_duplicate = []
    for i in range(len(list_unique_receipt[:2000])):
      list_year_unique = df_receipt_all.get_group(list_unique_receipt[i])['year'].unique().tolist()
      list_all_duplicate.append(f"{list_year_unique}")

    dict_sep_year = dict((i, list_all_duplicate.count(i)) for i in list_all_duplicate)

    list_unique_year = df_between_date['year'].unique().tolist()
    tuple_unique_year = tuple(df_between_date['year'].unique())

    list_set_year = []
    for i in range(len(list_unique_year)):
      list_set_year.append([list_unique_year[i]])
      for j in range(len(list_unique_year)):
        if i != j and i < j:
          list_set_year.append([list_unique_year[i],list_unique_year[j]])
          for k in range(len(list_unique_year)):
            if i != j and i !=k and j != k and i < k and i < j and j < k:
              list_set_year.append([list_unique_year[i],list_unique_year[j],list_unique_year[k]])
    list_set_year.sort(key = lambda x: x[0])
    list_set_year = sorted(list_set_year, key=len)
    for i in range(len(list_set_year)):
      try:
        dict_sep_year[f"{list_set_year[i]}"] = dict_sep_year[f"{list_set_year[i]}"]
      except:
        dict_sep_year[f"{list_set_year[i]}"] = 0
    # virtual
    if len(list_unique_year) == 3:
      v=venn3(subsets = (dict_sep_year[f"{list_set_year[0]}"], dict_sep_year[f"{list_set_year[1]}"], dict_sep_year[f"{list_set_year[2]}"], dict_sep_year[f"{list_set_year[3]}"],dict_sep_year[f"{list_set_year[4]}"],dict_sep_year[f"{list_set_year[5]}"],dict_sep_year[f"{list_set_year[6]}"]), set_labels = tuple_unique_year)
      plt.show()
    elif len(list_unique_year) == 2:
      venn2(subsets = (dict_sep_year[f"{list_set_year[0]}"], dict_sep_year[f"{list_set_year[1]}"], dict_sep_year[f"{list_set_year[2]}"]), set_labels = tuple_unique_year)
      plt.show()
    else:
      fig, ax = plt.subplots(figsize=(6, 10), subplot_kw=dict(aspect="equal"))
      recipe = list_set_year[0]
      wedges, texts, autotexts = ax.pie(recipe, autopct='{}'.format(dict_sep_year[f"{list_set_year[0]}"]),textprops=dict(color="w"))
      plt.setp(autotexts, size=8, weight="bold")
      ax.set_title("year {}".format(recipe[0]))
    # return plt.show()
    return df_between_date

print(virtualize_surburst_chart(df,'2018-01-01','2020-01-18'))


In [0]:
import plotly.graph_objects as go

fig = go.FigureWidget()

# # Create scatter trace of text labels
# fig.add_trace(go.Scatter(
#     x=[1, 1.75, 2.5],
#     y=[1, 1, 1],
#     text=["$A$", "$A+B$", "$B$"],
#     mode="text",
#     textfont=dict(
#         color="black",
#         size=18,
#         family="Arail",
#     )
# ))

# # Update axes properties
# fig.update_xaxes(
#     showticklabels=False,
#     showgrid=False,
#     zeroline=False,
# )

# fig.update_yaxes(
#     showticklabels=False,
#     showgrid=False,
#     zeroline=False,
# )

# # Add circles
# fig.add_shape(
#         go.layout.Shape(
#             type="circle",
#             fillcolor="blue",
#             x0=0,
#             y0=0,
#             x1=2,
#             y1=2,
#             line_color="blue"
#         ))
# fig.add_shape(
#         go.layout.Shape(
#             type="circle",
#             fillcolor="gray",
#             x0=1.5,
#             y0=0,
#             x1=3.5,
#             y1=2,
#             line_color="gray"
#         ))
# fig.update_shapes(dict(
#     opacity=0.3,
#     xref="x",
#     yref="y",
#     layer="below"
# ))
# # Update figure dimensions
# fig.update_layout(
#     margin=dict(
#         l=20,
#         r=20,
#         b=100
#     ),
#     height=600,
#     width=800,
#     plot_bgcolor="white"
# )

# fig.show()


In [0]:
def search_method(substring):
  object_methods = [method_name for method_name in dir(fig)]
  for method in object_methods:
    contains_subst = substring in method
    if contains_subst:
      print(method)

search_method('add')   